<h1>Process Data</h1>
<h4>Haley Johnson</h4>

<p>Notebook to extract, combine and transform data</p>

In [2]:
import pandas as pd
import numpy as np
import os 

<h2>Manipulate Happiness Data</h2>

In [30]:
html = pd.read_html("happiest_cities.html")

In [31]:
happiness_df = html[0]

In [34]:
happiness_df = happiness_df.rename(columns = {'City': 'Location'})

happiness_df['state'] = happiness_df['Location'].apply(lambda s: s.split(",")[-1])
happiness_df['city'] = happiness_df['Location'].apply(lambda s: s.split(",")[0])

happiness_df = happiness_df.drop(columns = ['Location'])

<h2>Manipulate Transportation Data</h2>

In [72]:
locations_df = pd.read_csv("EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv")

In [73]:
# still deciding what to do with names

# locations_df.rename(columns = {"GEOID10": "2010_fips_code", "GEOID20" : "2018_fips_code", 
#                     "STATEFP": 'state_fips_code', "COUNTYFP": "county_fips_code", 
#                     "Ac_Total": 'total_acres', 'Ac_Land': 'land_acres', 'Ac_Water': 'water_acres', 
#                     "Ac_Unpr": "unprotected_acres", "CountHU": "total_housing_units", "HH": "households", 
#                     "P_WrkAge": 'percent_working_age', 'AutoOwn0': 'owns_no_cars', 
#                     'Pct_AO0': 'percent_owns_no_cars', "AutoOwn1": 'owns_one_car', "Pct_AO1": "percent_owns_one_car",
#                     "AutoOwn2p": "owns_two_plus_cars", "Pct_AO2": 'percent_owns_2_plus_cars'})

<h3>Remove Rows Where Location Is Missing</h3>

In [74]:
locations_df = locations_df[pd.isnull(locations_df['CBSA_Name']) == False]

<h3>Get City and State Names</h3>

In [75]:
locations_df['CBSA_Name'] = locations_df['CBSA_Name'].astype(str)

In [76]:
locations_df['metro_area_cities'] = locations_df['CBSA_Name'].apply(lambda s: str(s).split(",")[0])
locations_df['metro_area_states'] = locations_df['CBSA_Name'].apply(lambda s: str(s).split(",")[-1])

locations_df['city'] = locations_df['metro_area_cities'].apply(lambda s: s.split("-"))
locations_df['state'] = locations_df['metro_area_states'].apply(lambda s: s.split("-"))

In [77]:
locations_df = locations_df.drop(columns = ['metro_area_cities', 'metro_area_states'])

<h3>Explode</h3>

In [94]:
exploded = locations_df.explode('city')
exploded = exploded.explode('state')
exploded = exploded.drop_duplicates()

<h2>Merging</h2>

In [ ]:
test = exploded.head().merge(happiness_df, right_on = ['state', 'city'])

In [98]:
happiness_df.head(left)

,Overall Rank,Total Score,Emotional & Physical Well-Being,Income & Employment,Community & Environment,state,city
0,1,77.13,1,46,2,CA,Fremont
1,2,71.64,3,161,5,MD,Columbia
2,3,71.15,4,1,44,CA,San Francisco
3,4,71.00,2,6,55,CA,San Jose
4,5,70.38,5,16,12,CA,Irvine


<h3>Old Approach to Exploding — maybe delete later</h3>

Credit to <a href = "https://stackoverflow.com/questions/45846765/efficient-way-to-unnest-explode-multiple-list-columns-in-a-pandas-dataframe">this</a> Stack Overflow post for helping me figure out how to do a multicolumn explode. 

In [78]:
cols = [c for c in list(locations_df.columns) if c not in ['city', 'state']]

In [89]:
multi_index = locations_df.set_index(cols)
multi_index['row'] = np.arange(len(multi_index))
multi_index = multi_index.set_index(['row'], append = True)

In [92]:
multi_index.head()

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,city,state
OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,CBSA_Name,CBSA_POP,CBSA_EMP,CBSA_WRK,Ac_Total,Ac_Water,Ac_Land,Ac_Unpr,TotPop,CountHU,HH,P_WrkAge,AutoOwn0,Pct_AO0,AutoOwn1,Pct_AO1,AutoOwn2p,Pct_AO2p,Workers,R_LowWageWk,R_MedWageWk,R_HiWageWk,R_PCTLOWWAGE,TotEmp,E5_Ret,E5_Off,E5_Ind,E5_Svc,E5_Ent,E8_Ret,E8_off,E8_Ind,E8_Svc,E8_Ent,E8_Ed,E8_Hlth,E8_Pub,E_LowWageWk,E_MedWageWk,E_HiWageWk,E_PctLowWage,D1A,D1B,D1C,D1C5_RET,D1C5_OFF,D1C5_IND,D1C5_SVC,D1C5_ENT,D1C8_RET,D1C8_OFF,D1C8_IND,D1C8_SVC,D1C8_ENT,D1C8_ED,D1C8_HLTH,D1C8_PUB,D1D,D1_FLAG,D2A_JPHH,D2B_E5MIX,D2B_E5MIXA,D2B_E8MIX,D2B_E8MIXA,D2A_EPHHM,D2C_TRPMX1,D2C_TRPMX2,D2C_TRIPEQ,D2R_JOBPOP,D2R_WRKEMP,D2A_WRKEMP,D2C_WREMLX,D3A,D3AAO,D3AMM,D3APO,D3B,D3BAO,D3BMM3,D3BMM4,D3BPO3,D3BPO4,D4A,D4B025,D4B050,D4C,D4D,D4E,D5AR,D5AE,D5BR,D5BE,D5CR,D5CRI,D5CE,D5CEI,D5DR,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area,row,,
1,4.811300e+11,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,"Dallas-Fort Worth-Arlington, TX",7189384,3545715,3364458,73.595028,0.0,73.595028,73.595028,1202,460.0,423.0,0.549,69,0.163121,39,0.092199,315,0.744681,412,99,122,191,0.240291,66,20,3,0,19,24,20,3,0,15,24,0,4,0,21,27,18,0.318182,6.250422,16.332625,0.896800,0.271757,0.040764,0.000000,0.258170,0.326109,0.271757,0.040764,0.000000,0.203818,0.326109,0.000000,0.054351,0.0,7.147222,0,0.156028,0.886264,0.763386,0.855442,0.662091,0.348912,0.526296,0.585916,0.287128,0.104101,0.276151,6.242424,5.287423e-03,23.534900,0.000000,10.655277,12.879623,115.981747,0.000000,60.873677,8.696240,34.784958,43.481198,362.10,0.0,0.000000,4.33,37.654717,0.003602,433601,303660,135362,53504,0.000398,0.785893,0.000358,0.841299,0.000525,0.184697,0.000476,0.137707,6,14,15,17,14.000000,3110.360820,297836.0831,0,"[Dallas, Fort Worth, Arlington]",[ TX]
2,4.811300e+11,4.811300e+11,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,"Dallas-Fort Worth-Arlington, TX",7189384,3545715,3364458,119.829909,0.0,119.829909,119.214200,710,409.0,409.0,0.466,0,0.000000,168,0.410758,241,0.589242,395,76,107,212,0.192405,25,7,0,3,15,0,7,0,3,13,0,0,2,0,10,4,11,0.400000,3.430799,5.955666,0.209707,0.058718,0.000000,0.025165,0.125824,0.000000,0.058718,0.000000,0.025165,0.109047,0.000000,0.000000,0.016777,0.0,3.640506,0,0.061125,0.835015,0.569986,0.831686,0.554458,0.197047,0.248481,0.271309,0.002033,0.068027,0.119048,15.800000,3.736300e-07,22.893366,0.755137,2.859482,19.278747,80.145600,5.340904,10.681807,10.681807,85.454458,5.340904,718.84,0.0,0.009516,4.33,23.126113,0.006099,386504,272135,236885,90089,0.000355,0.700531,0.000321,0.753958,0.000919,0.323221,0.000801,0.231868,3,10,12,14,10.833333,3519.469110,484945.1466,1,"[Dallas, Fort Worth, Arlington]",[ TX]
3,4.811300e+11,4.811300e+11,48,113,7825,3,206.0,"Dallas-Fort Worth, TX-OK",19100.0,"Dallas-Fort Worth-Arlington, TX",7189384,3545715,3364458,26.367053,0.0,26.367053,26.367050,737,365.0,329.0,0.811,19,0.057751,143,0.434650,167,0.507599,463,136,189,138,0.293737,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,13.843035,27.951553,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,13.843035,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.367879,0.000000,1.000000,0.000000,0.000000e+00,14.210509,6.128365,2.611430,5.470714,24.272717,24.272717,0.000000,24.272717,0.000000,0.000000,398.31,0.0,0.000000,3.00,72.818150,0.004071,404573,288925,230587,82815,0.000371,0.733281,0.000340,0.800475,0.000894,0.314628,0.000736,0.213146,1,1,7,17,8.333333,1697.091802,106705.9281,2,"[Dallas, Fort Worth, Arlington]",[ TX]
4,4.811300e+11,4.811300e+11,48,113,7824,1,206.0,"Dallas-Fort Worth, TX-OK",19100.0,"Dallas-Fort Worth-Arlington, TX",7189384,3545715,3364458,119.060687,0.0,119.060687,119.060687,904,384.0,384.0,0.638,0,0.00

In [91]:
locations_df_long = multi_index.apply(pd.Series.explode).reset_index()

ValueError: cannot handle a non-unique multi-index!